In [1]:
import torch 
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor()])


train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

100.0%
100.0%
100.0%
100.0%


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class MNISTModel(nn.Module):
    def __init__(self) -> None:
        super(MNISTModel, self).__init__()
        self.fc1 = nn.Linear(in_features=28*28, out_features=128) 
        self.fc2 = nn.Linear(in_features=128, out_features=10)    

    def forward(self, x):# -> Any:
        x = x.view(-1, 28*28)  # flatten image
        x = F.relu(input=self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
model = MNISTModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(5):  # train for 5 epochs
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/5], Loss: {loss.item():.4f}")

Epoch [1/5], Loss: 0.2316
Epoch [2/5], Loss: 0.0909
Epoch [3/5], Loss: 0.0387
Epoch [4/5], Loss: 0.3710
Epoch [5/5], Loss: 0.0840


In [4]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 97.28%
